In [1]:
import joblib
import json
from copy import copy
import importlib
import os
import sys
import numpy as np
import rpy2.robjects as robjects
import pandas as pd
robjects.r('.sourceAlfunction()')
print(sys.path[0])
sys.path.insert(0, os.path.abspath(os.path.join(sys.path[0], 'NEXT/4EBaseMetal/code/utils')))
read_data = importlib.import_module("read_data")
construct_data = importlib.import_module("construct_data")
transform_data = importlib.import_module("transform_data")
normalize_feature = importlib.import_module("normalize_feature")

 

[1]
 "SinglePackageAl loaded. Attaching version 19.04.6"


[1]
 "Loaded AlphienData"


[1]
 "Loaded AlphienTechnicalAnalysis"


[1]
 "Loaded AlphienFundamental"


[1]
 "Loaded AlphienBaseMetals"


[1]
 "Loaded AlphienOption"


send2Log: Your working directory is: /home/chanmingwei
 





In [12]:
def h5_function(date_range,ground_truth):
    h = 5
    time_series = None
    rgetSecurity = robjects.r('getSecurity')
    rindex = robjects.r('index')
    rgetGenOHLCV = robjects.r('getGenOHLCV')
    gt_data = rgetSecurity(securityName=ground_truth,zoom=date_range)
    gt_index = rindex(gt_data)
    extra_data = None
    extra_index = None
    if ground_truth == "LMAHDY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("ZNAcl",zoom = \"'+date_range+'\")[,6]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMPBDY Comdty":
        lag = 20
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("AAcl",zoom = \"'+date_range+'\")[,6]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMNIDY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("PA",zoom = \"'+date_range+'\")[,1]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMZSDY Comdty":
        lag = 20
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getSecurityOHLCV("LMNIDS03 Comdty", zoom = \"'+date_range+'\")[,3]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMSNDY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getGenOHLCV("RTcl",zoom = \"'+date_range+'\")[,6]')
        extra_index = rindex(extra_data)
    elif ground_truth == "LMCADY Comdty":
        lag = 40
        norm_ex = "v1"
        norm_volume = "v1"
        norm_3m_spread = "v1"
        extra_data = robjects.r('getSecurityOHLCV("LMCADS03 Comdty",zoom = \"'+date_range+'\")[,6]')
        extra_index = rindex(extra_data)
    else:
        print("ground truth val out of bounds!")
        return None
    gt_data_list = []
    gt_index_list = []
    extra_data_list = []
    extra_index_list = []
    for i in range(len(gt_index)):
        gt_index_list.append(gt_index[i])
    for i in range(len(gt_data)):
        gt_data_list.append(gt_data[i])
    for i in range(len(extra_index)):
        extra_index_list.append(extra_index[i])
    for i in range(len(extra_data)):
        extra_data_list.append(extra_data[i])
    start_date= min(gt_index_list[0],extra_index_list[0])
    end_date =max(gt_index_list[-1],extra_index_list[-1])
    if ground_truth == "LMAHDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Al_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'SHFE_Zi_OI':extra_data_list}
    elif ground_truth == "LMPBDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Le_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'SHFE_AA_OI':extra_data_list}
    elif ground_truth == "LMNIDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Ni_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list+86400*np.ones(len(extra_index_list)), 'COMEX_PA_lag1_Open':extra_data_list}
        extra_dict = None
    elif ground_truth == "LMZSDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Zi_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'LME_Ni_Low':extra_data_list}
    elif ground_truth == "LMSNDY Comdty":
        gt_dict={'time':gt_index_list,'LME_Ti_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'SHFE_RT_OI':extra_data_list}
    elif ground_truth == "LMCADY Comdty":
        gt_dict={'time':gt_index_list,'LME_Co_Spot':gt_data_list}
        extra_dict = {'time':extra_index_list, 'LME_Co_OI':extra_data_list}
    else:
        print("ground truth val out of bounds!")
        return None
    gt_dataframe = pd.DataFrame(gt_dict)
    if extra_dict is not None:
        extra_dataframe = pd.DataFrame(extra_dict)
    #     print(isinstance(extra_dataframe, pd.DataFrame))
        time_series = pd.DataFrame.merge(gt_dataframe,extra_dataframe,how ='outer', on = 'time')
    else:
        time_series = gt_dataframe
    time_series.set_index('time')
#     print(time_series['time'])
    time_series = time_series.drop('time',axis = 1)
    

#     print(time_series)
    
    #Processing data
    time_series = read_data.process_missing_value_v3(time_series,np.min([lag,10]))
    org_cols = time_series.columns.values.tolist()
    print("Normalizing")

    #Normalize and generate technical indications
    norm_params = construct_data.normalize(time_series,vol_norm = norm_volume, spot_spread_norm=norm_3m_spread,ex_spread_norm = norm_ex)
    time_series = copy(norm_params["val"])
    del norm_params["val"]
    time_series = construct_data.technical_indication(time_series)
    cols = time_series.columns.values.tolist()
    for col in cols:
        if "_Volume" in col or "_OI" in col or "CNYUSD" in col:
            time_series = time_series.drop(col,axis = 1)
            org_cols.remove(col)
    print(time_series.columns.values.tolist())
    curr = ""
    if ground_truth == "LMAHDY Comdty":
        curr = "Al"
    elif ground_truth == "LMPBDY Comdty":
        curr = "Le"
    elif ground_truth == "LMNIDY Comdty":
        curr = "Ni"
    elif ground_truth == "LMZSDY Comdty":
        curr = "Zi"
    elif ground_truth == "LMSNDY Comdty":
        curr = "Ti"
    elif ground_truth == "LMCADY Comdty":
        curr = "Co"
    else:
        print("ground truth val out of bounds!")
        return None
#     print(time.series.columns.values)
    norm_data = copy(normalize_feature.log_1d_return(time_series,org_cols))
    norm_data = read_data.process_missing_value_v3(norm_data,10)
    to_be_predicted = norm_data["LME_"+curr+"_Spot"]
    if h > 1:
        for i in range(h-1):
            to_be_predicted = to_be_predicted + norm_data["LME_"+curr+"_Spot"].shift(-i-1)
    gt = (to_be_predicted > 0).shift(-1)

    start_index = 0
    if start_index < lag - 1:
        start_index = lag - 1
    end_index = len(norm_data)-1
    assert end_index >= lag - 1

    X_te, y_te = construct_data.construct(norm_data, gt, start_index, end_index, lag, "log_1d_return")
    X_te = transform_data.flatten(X_te)
    y_te = y_te*2 - 1

    #load model
    model = joblib.load("NEXT/LME_"+curr+"_Spot_h"+str(h)+"_n2.joblib")

    prediction = model.predict(X_te).reshape(X_te.shape[0],1)


    total_no = prediction.shape[0]
    no_true = sum(np.equal(prediction,y_te))
    no_TT = sum(np.multiply(prediction+1,y_te+1))/4
    no_FF = sum(np.multiply(prediction - 1,y_te - 1))/4
    no_TF = -sum(np.multiply(prediction + 1,y_te - 1))/4
    no_FT = -sum(np.multiply(prediction - 1,y_te + 1))/4

    #some basic statistics, to identify whether our model is more prone to which type of error.
    print("Overall Accuracy:%d",no_true/total_no )
    print("TT:%d", no_TT)
    print("TF:%d", no_TF)
    print("FT:%d", no_FT)
    print("FF:%d", no_FF)
    return prediction

In [3]:
LMAHDY = h5_function("2015-01-06::2016-01-05","LMAHDY Comdty")

send2Log: Fetching generic for ZNA1C
 




/opt/anaconda2/envs/gpustudio-torch/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: 'mondate'


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/envs/gpustudio-torch/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from 'package:magrittr':

    add, subtract


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/envs/gpustudio-torch/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following object is masked from 'package:base':

    as.difftime


  warnings.warn(x, RRuntimeWarning)


Normalizing
Normalizing OI:SHFE_Zi_OI=>SHFE_Zi_nOI
['LME_Al_Spot', 'SHFE_Zi_nOI']
Overall Accuracy:%d [0.50505051]
TT:%d [51.]
TF:%d [59.]
FT:%d [39.]
FF:%d [49.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
LMCADY = h5_function("2015-01-06::2016-01-05","LMCADY Comdty")

Normalizing
Normalizing OI:LME_Co_OI=>LME_Co_nOI
['LME_Co_Spot', 'LME_Co_nOI']
Overall Accuracy:%d [0.50943396]
TT:%d [45.]
TF:%d [68.]
FT:%d [36.]
FF:%d [63.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
LMNIDY = h5_function("2015-01-06::2016-01-05","LMNIDY Comdty")

send2Log: Fetching generic for PA1I
 


Normalizing
['LME_Ni_Spot']
Overall Accuracy:%d [0.53051643]
TT:%d [24.]
TF:%d [40.]
FT:%d [60.]
FF:%d [89.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
LMPBDY = h5_function("2015-01-06::2016-01-05","LMPBDY Comdty")

send2Log: Fetching generic for AA3C
 


Normalizing
Normalizing OI:SHFE_AA_OI=>SHFE_AA_nOI
['LME_Le_Spot', 'SHFE_AA_nOI']
Overall Accuracy:%d [0.4587156]
TT:%d [67.]
TF:%d [93.]
FT:%d [25.]
FF:%d [33.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
LMSNDY = h5_function("2015-01-06::2016-01-05","LMSNDY Comdty")

send2Log: Fetching generic for RT1C
 


Normalizing
Normalizing OI:SHFE_RT_OI=>SHFE_RT_nOI
['LME_Ti_Spot', 'SHFE_RT_nOI']
Overall Accuracy:%d [0.55050505]
TT:%d [62.]
TF:%d [70.]
FT:%d [19.]
FF:%d [47.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
LMZSDY = h5_function("2015-01-06::2016-01-05","LMZSDY Comdty")

Normalizing
['LME_Zi_Spot', 'LME_Ni_Low']
Overall Accuracy:%d [0.48927039]
TT:%d [50.]
TF:%d [91.]
FT:%d [28.]
FF:%d [64.]


/home/chanmingwei/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
